In [ ]:
The topChef database
TopChef is a reality TV show in which contestants, who are professional chefs, compete in a
variety of cooking contests over the course of a season. Each episode, one or more chefs are
kicked off the show, until, at the end the winner is crowned “Top Chef”.
The topChef database contains information about (most of) the top Chef seasons.
You can access the top chef database using the same credentials we used in class for the
sakila database. The only difference is that the database name (dbname) is topChef instead
of sakila.
If you would prefer to work in R (and even avoid SQL), you can load the topChef data by
install.packages("topChef") and library("topChef").
There are 6 tables in the topChef database but we will focus our attention on two of them:
chefdetails and judges. The chefdetails tells us information about the contestants, and
the judges table tells us information about the judges.
Of particular interest in the chefdetails table are the fields:
• season which is the name of the season (often a city where the competition took place, but not always)
• seasonNumber which is the number of the season
• name which is the chef’s name 3
• placement which tells where the chef finished in the competition
• gender which tells the chef’s gender.

In the judges table we are interested in:
• season and seasonNumber which are the same as in the chefdetails table.
• episode which gives the episode where the judge appeared; the show uses many guest
judges varying episode by episode.
• challengeType describes the type of challenge where the judge played a role.
• guestJudge is the judge’s name
• competedOnTC is ‘Yes’ if the judge was a previous top chef contestant.

In [101]:
#
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

username = "user"  
password = "grad5100user"  
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"  
port = 3306

In [102]:
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)


In [103]:
def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result


In [104]:
tables = run_sql("show tables;", engine)
for x in tables:
    print(x)

('challengedescriptions',)
('challengewins',)
('chefdetails',)
('episodeinfo',)
('judges',)
('rewards',)


In [105]:
#Problem One

chefdetails_query = """SELECT * FROM chefdetails WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6' """
filtered_chefdetails = df_select(chefdetails_query, engine)
print(filtered_chefdetails)

judges_query = """SELECT * FROM judges WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6' """
filtered_judges = df_select(judges_query, engine)
print(filtered_judges)


Retrieved 322 records
     num                       name                       chef  \
0      0              Richard Blais                 Richard B.   
1      1              Mike Isabella                    Mike I.   
2      2             Antonia Lofaso                 Antonia L.   
3      3              Tiffany Derry                 Tiffany D.   
4      4                 Carla Hall                   Carla H.   
..   ...                        ...                        ...   
317  386  May Phattanant Thongthong  May Phattanant Thongthong   
318  387             Begona Rodrigo             Begona Rodrigo   
319  388          Gabriel Rodriguez          Gabriel Rodriguez   
320  389               Amar Santana                    Amar S.   
321  390            Sylwia Stachyra            Sylwia Stachyra   

                        hometown       city       state   age  \
0                           None       None        None  38.0   
1                           None       None        None

In [98]:
#Problem Two 

chefdetails_query = """SELECT season, seasonNumber, name, placement, gender FROM chefdetails WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6' """
filtered_chefdetails = df_select(chefdetails_query, engine)
print(filtered_chefdetails)

Retrieved 322 records
                  season  seasonNumber                       name  placement  \
0    All Stars: New York             8              Richard Blais          1   
1    All Stars: New York             8              Mike Isabella          2   
2    All Stars: New York             8             Antonia Lofaso          3   
3    All Stars: New York             8              Tiffany Derry          4   
4    All Stars: New York             8                 Carla Hall          5   
..                   ...           ...                        ...        ...   
317      World All Stars            20  May Phattanant Thongthong         14   
318      World All Stars            20             Begona Rodrigo         12   
319      World All Stars            20          Gabriel Rodriguez          2   
320      World All Stars            20               Amar Santana          6   
321      World All Stars            20            Sylwia Stachyra         11   

     gender  
0  

In [99]:
#Problem 2

judges_query = """SELECT season, seasonNumber, episode, challengeType, guestJudge, competedOnTC FROM judges WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6' """
filtered_judges = df_select(judges_query, engine)
print(filtered_judges)

Retrieved 633 records
                  season  seasonNumber  episode challengeType  \
0    All Stars: New York             8        1     Quickfire   
1    All Stars: New York             8        1   Elimination   
2    All Stars: New York             8        2     Quickfire   
3    All Stars: New York             8        2   Elimination   
4    All Stars: New York             8        3     Quickfire   
..                   ...           ...      ...           ...   
628      World All Stars            20       12     Quickfire   
629      World All Stars            20       12   Elimination   
630      World All Stars            20       13     Quickfire   
631      World All Stars            20       13   Elimination   
632      World All Stars            20       14   Elimination   

           guestJudge competedOnTC  
0       Tom Colicchio         None  
1    Anthony Bourdain         None  
2           Joe Jonas         None  
3           Katie Lee         None  
4         Da

In [100]:
distinct_season = """SELECT DISTINCT season, seasonNumber FROM chefdetails WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6' ORDER BY seasonNumber"""
season_map = df_select(distinct_season, engine)
print(season_map)

Retrieved 20 records
                 season  seasonNumber
0         San Francisco             1
1           Los Angeles             2
2                 Miami             3
3               Chicago             4
4              New York             5
5             Las Vegas             6
6                  D.C.             7
7   All Stars: New York             8
8                 Texas             9
9               Seattle            10
10          New Orleans            11
11               Boston            12
12           California            13
13           Charleston            14
14             Colorado            15
15             Kentucky            16
16       All-Stars L.A.            17
17             Portland            18
18              Houston            19
19      World All Stars            20


In [73]:
#a. Among all chef contestants, how many are male and how many are female? There are 170 males and 152 females

gender_count = filtered_chefdetails['gender'].value_counts()
print(gender_count)

gender
Male      170
Female    152
Name: count, dtype: int64


In [74]:
#b. Among all winners (placement==1) how many are male and how many are female? 14 males and 6 females.

winners_df = filtered_chefdetails[filtered_chefdetails['placement'] == 1]
winners_gender_count = winners_df['gender'].value_counts()
print(winners_gender_count)


gender
Male      14
Female     6
Name: count, dtype: int64


In [75]:
#c. Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many are female? 36 males and 25 females.

top3_finalists_df = filtered_chefdetails[filtered_chefdetails['placement'].isin([1, 2, 3])]
top3_finalists_gender_count = top3_finalists_df['gender'].value_counts()
print(top3_finalists_gender_count)


gender
Male      36
Female    25
Name: count, dtype: int64


In [76]:
#d. Compare the number of times a female was placed in the top 3 with the number of times a female won. Does this number seem unusual? That is, are women who place in the top 3 less likely than expected to ultimately win the competition? Why or why not?

female_top3_count = len(filtered_chefdetails[(filtered_chefdetails['gender'] == 'Female') & (filtered_chefdetails['placement'].isin([1, 2, 3]))])
female_winner_count = len(filtered_chefdetails[(filtered_chefdetails['gender'] == 'Female') & (filtered_chefdetails['placement'] == 1)])

print('Amount of Female placed in Top 3:', female_top3_count)
print('Amount of Female placed in first:', female_winner_count)
print('Percentage of female (first place) from top 3:', (female_winner_count/female_top3_count)*100)

#The number does seem unusual because it's not a high percentage that women got placed in first place compared. This shows that top 3 does not guarantee first place.

Amount of Female placed in Top 3: 25
Amount of Female placed in first: 6
Percentage of female (first place) from top 3: 24.0


In [88]:
#Make a table with one row for each first place winner containing the winner’s name, the season they won, and a column containing ‘Yes’ or ‘No’ depending on whether they served as a guest judge or not.
first_place_df = """
SELECT
  cd.name,
  cd.season,
  IF((SELECT 1 FROM judges WHERE guestJudge = cd.name OR competedOnTC = cd.name LIMIT 1), 'Yes', 'No') AS served_as_guest_judge
FROM
  chefdetails cd
WHERE
  cd.placement = 1
  AND cd.season NOT LIKE '%Masters%'
  AND cd.season != 'Canada 6';
"""
table = df_select(first_place_df, engine)
print(table)


Retrieved 20 records
                    name               season served_as_guest_judge
0          Richard Blais  All Stars: New York                   Yes
1           Melissa King       All-Stars L.A.                   Yes
2                Mei Lin               Boston                    No
3            Jeremy Ford           California                    No
4      Brooke Williamson           Charleston                   Yes
5        Stephanie Izard              Chicago                   Yes
6           Kevin Sbraga                 D.C.                    No
7           Joseph Flamm             Colorado                    No
8             Kah-wai Lo              Houston                    No
9   Kelsey Barnard Clark             Kentucky                   Yes
10     Michael Voltaggio            Las Vegas                   Yes
11             Ilan Hall          Los Angeles                   Yes
12            Hung Huynh                Miami                   Yes
13         Nicholas Elmi   

In [92]:
#Find all contestants who were not first place winners but did serve as a guest judge at some point. Make sure to only list each name once.
non_winner = """SELECT DISTINCT
  cd.name AS non_winner_name,
  'Yes' AS served_as_guest_judge
FROM
  chefdetails cd
WHERE
  cd.placement != 1
  AND cd.season NOT LIKE '%Masters%'
  AND cd.season != 'Canada 6'
  AND EXISTS (
    SELECT 1 FROM judges
    WHERE guestJudge = cd.name OR competedOnTC = cd.name
  );
"""

n_winner = df_select(non_winner, engine)
print(n_winner)

Retrieved 32 records
       non_winner_name served_as_guest_judge
0        Mike Isabella                   Yes
1       Antonia Lofaso                   Yes
2        Tiffany Derry                   Yes
3           Dale Talde                   Yes
4     Spike Mendelsohn                   Yes
5     Stephen Asprinio                   Yes
6      Bryan Voltaggio                   Yes
7      Gregory Gourdet                   Yes
8      Karen Akunowicz                   Yes
9        Lee Anne Wong                   Yes
10         Nini Nguyen                   Yes
11        Melissa King                   Yes
12        Amar Santana                   Yes
13      Kwame Onwuachi                   Yes
14      Sheldon Simeon                   Yes
15          Sam Talbot                   Yes
16       Richard Blais                   Yes
17   Adrienne Cheatham                   Yes
18        Carrie Baird                   Yes
19     Michael Midgley                   Yes
20        Betty Fraser            